In [15]:
import pandas as pd
import numpy as np
import pickle as pkl
import scipy.io as sio

In [16]:
path_old='/scicore/home/doetsch/GROUP/scigrp/vargasSingleCell3/'


datafile = path_old+'OUTPUT/concat_mat/prom_expr_matrix.mtx'
data = sio.mmread(datafile).toarray().astype(np.int64)#.transpose()
genes = pd.read_csv(path_old+'OUTPUT/concat_mat/prom_expr_promoters.tsv', header=None, sep="\t")
cells = pd.read_csv(path_old+'OUTPUT/concat_mat/accepted_barcodes.tsv', header=None, sep="\t" )

df_o = pd.DataFrame(data,index=genes[0],columns=cells[0])

path='/scicore/home/doetsch/GROUP/scigrp/vargasSingleCell3_2males_added/'

datafile = path+'OUTPUT/concat_mat/prom_expr_matrix.mtx'
data = sio.mmread(datafile).toarray().astype(np.int64)#.transpose()
genes = pd.read_csv(path+'OUTPUT/concat_mat/prom_expr_promoters.tsv', header=None, sep="\t")
cells = pd.read_csv(path+'OUTPUT/concat_mat/accepted_barcodes.tsv', header=None, sep="\t" )

df_n = pd.DataFrame(data,index=genes[0],columns=cells[0])

df = pd.concat([df_o,df_n],axis=1).fillna(0)
data = df.to_numpy().astype(int)

alpha = 2**(np.round(np.log2(data.sum()/data.shape[1])))
LAMBDA = alpha*np.sum(data, axis=1)/np.sum(data)
mask = LAMBDA > 0
data = data[mask, :]
LAMBDA = LAMBDA[mask]
genes = df.index[mask]
df = pd.DataFrame(data,index=genes,columns=df.columns)

clusters = np.loadtxt(path+'optimized_clusters.txt', dtype=np.int64)
lmbd = np.loadtxt(path+'dirichlet_pseudocounts.txt')
hierarchy_df = pd.read_csv(path+'cluster_hierarchy.tsv', sep='\t')
score_df = pd.read_csv(path+'marker_gene_scores.tsv', sep='\t')
score_df = pd.concat([hierarchy_df, score_df],axis=1)
annotation = np.array(np.loadtxt(path+'batches.txt', dtype='S12'),dtype=str)
print(np.unique(annotation))
n_scale = np.median(data.sum(axis=0))

['Adam' 'Ana' 'Eve' 'Fiona' 'John' 'Melvin' 'Tom' 'Viole']


In [17]:
pklfile=open("varagasSingleCell3_2males_added.pkl",'wb')
pkl.dump(df,pklfile)
pkl.dump(clusters,pklfile)
pkl.dump(hierarchy_df,pklfile)
pkl.dump(score_df,pklfile)
pkl.dump(annotation,pklfile)
pkl.dump(n_scale,pklfile)
pkl.dump(lmbd,pklfile)
pklfile.close()